# CPython data model a správa paměti

## Pod kapotou

V této části si jen orientačně naznačíme, jakým způsobem jsou objekty v Pythonu vlastně implementované.

Každý objekt v Pythonu je ve skutečnosti napsán, jako struktura v jazyce C. Struktury nejsou nic jiného než složený datový typ, který sdružuje více proměnných pod jedním jménem. Konkrétně jsou všechny objekty něakou variací na strukturu `PyObject`. Z pohledu OOP jsou všechny objekty v Pythonu potomkem jednoho společného předka, a sice objektu `Object`. V Pythonu 2 bylo dokonce nutné při definici vlastní třídy tento vztah explicitně zdůrazňovat (viz kapitola o dědičnosti).

```python
class MyClass(Object):
    ...
```

V těchto C strukturách jsou uloženy všechny možné informace: informace o typu, počet referencí (uvidíme u správy paměti), seznamy s funkcemi na těchto strukturách definovanými atd. CPython intenzivně využívá pointer type casting (tj. předstírá, že pointer na strukturu v paměti ve skutečnosti ukazuje na jiný typ struktury), díky čemuž je možné implementovat takovou funkcionalitu, jako je například polymorphismus, ale za cenu toho, že výsledný kód je poměrně zdlouhavý a obsahuje velikou spoustu všelijakých kontrol. Přikládám úryvek z dokumentace

> **Note** The explicit cast to destructor above is needed because we defined `Custom_dealloc` to take a `CustomObject *` argument, but the `tp_dealloc` function pointer expects to receive a `PyObject *` argument. Otherwise, the compiler will emit a warning. **This is object-oriented polymorphism, in C!**

My se tímto směrem ještě jednou vydáme a ukážeme si explicitně, jak v CPythonu vypadá sčítání dvou čísel, ale prozatím nám toto stačí.

## Správa paměti

V jazycích, jako je C, je nutné ručně řídit správu paměti, obzvlášť v případech, kdy např. velikost používaných polí není známa během compile time, ale až během run time. To obvykle probíhá ve dvou krocích - alokace a dealokace. Následující příklad v C alokuje paměť pro pole celých čísel.

```c
int some_function(int n) {
    int * array = malloc(sizeof(int) * n);

    if (array == NULL)
        return SOME_ERROR_CODE;

    process_array(array, n);
    
    free(array);
}
```
Manuální správa paměti nám sice dává velkou kontrolu, ale je velmi častým zdrojem chyb. Typickou chybou bývá opomenutí `free`, což znamená, že po skončení programu nebyla všechna paměť navrácena systému, tzv. _memory leak error_. Jinou častou chybou je opomenutí kontroly, zda alokace skutečně proběhla. Snažíme-li se pak např. zapisovat do této nealokované paměti, setkáme se s nechvalně známou _segmentation fault_, která už od low level programování odradila mnoho lidí.

V jazycích jako je Python, Java, JavaScript atd. se využívá naopak automatické správy paměti metodou, které říkáme garbage collection. Jako programátor se tedy od alokaci a dealokaci nestaráme a garbage collector to zajistí za nás - nevyužívané či nepřístupné objekty dekonstruuje a jim náležící paměť navrátí do poolu (Python si alokuje něco, čemu říká memory arenas, typicky po 256 KB, které dále pools po 4 KB. Tyto pools slouží pro alokaci objektů podobné velikosti, s cílem zamezit fragmentaci. V jednotlivých pools je paměť pro objekty alokována po blocích) 

Python implementuje garbage collection pomocí reference counting - počítání referencí. Dokud je objekt nenulový počet referencí, zůstává v paměti, jakmile počet referencí klesne na 0, tj. na náš objekt nic neukazuje, je nedosažitelný, garbage collector ho smaže a pamět uvolní. Můžeme si to vyzkoušet pomocí modulu `sys` a zcela generického objektu.

In [6]:
import sys

x = object()
print(sys.getrefcount(x))
y = x
print(sys.getrefcount(x))
del x
print(sys.getrefcount(y))

2
3
2


Otázka: proč to např. pro x=1 dopadne takhle?

In [7]:
import sys

x = 1
sys.getrefcount(x)

3529

## Generace

Python aplikuje něco, čemu se občas říká _Generation hypothesis_, tedy předpoklad, že většina objektů zahyne krátce po svém vzniku. Pokud náhodou nepřežije, pak se přesune do další generace. Pythoní GC udržuje 3 generace objektů - Generation 0, 1, 2, někdy nazývané young, middleaged a old.

GC zahají sběr - postupně prochází všechny objekty a pokud mají 0 referencí, poznačí si je na seznam k likvidaci. Pokud mají více referencí a jsou tedy dosažitelné, přesune je do starší generace.

A kdy se vlastně sběr iniciuje? to je dáno počtem objektů v každé generaci. Každá generace má definovaný threshold, který když je překročen, proběhne sběr v dané generaci. To si můžeme prohlédnout prostřednictvím modul `gc`. Funkce `get_threshold()` nám vrátí aktuální threshold pro jednotlivé generace. Hodnoty můžeme měnit pomocí funkce `set_threshold(young, mid, old)`, ale obecně platí, že na chod GC bychom spíše neměli sahat, pokud k tomu nemám opravdu dobrý důvod. Zatím jsem nikdy neměl ani špatný důvod na chod GC sahat.

In [22]:
import gc
# gc.set_threshold(1000, 15, 15)
gc.get_threshold()

(1000, 15, 15)

Aktuální počet objektů v každé generaci si můžeme zobrazit a můžeme se rozhodnout i iniciovat kolekci manuálně.

In [24]:
import gc

print(gc.get_count())
gc.collect()
print(gc.get_count())

(734, 12, 3)
(26, 0, 0)


## Self reference a cykly

Problém nastane, když se v referencí objeví cyklus. Ukažme si jednoduchý příklad:

In [18]:
import sys

x = []
x.append(x)

print(sys.getrefcount(x))
del x

3


Po vymázání proměnné x stále zůstává jedna aktivní reference - první prvek listu referuje k listu. Pokud by GC kontroloval jen počet referencí, list `x` by v paměti zůstal a nebylo by ho jak odstranit. Pythoní algoritmus na detekci cyklů je popsán zde: https://devguide.python.org/internals/garbage-collector/
Zde to zatím rozebírat nebudu.